In [1]:
import sys
sys.path.append("..")
import msgspec
import faker
from tqdm.auto import tqdm
from typing import Any, cast
import pickle
import gc
import random
import datetime

In [2]:
fake = faker.Faker("ru_RU")

In [3]:
from dkvs.bptree import BPTree
BPTree.HALF_INODE_SIZE = 32
BPTree.MAX_INODE_SIZE = BPTree.HALF_INODE_SIZE * 2
BPTree.HALF_LEAF_SIZE = 128
BPTree.MAX_LEAF_SIZE = BPTree.HALF_LEAF_SIZE * 2

In [4]:
class MyKey(msgspec.Struct, frozen=True, order=True):
    full_name: str

class MyValue(msgspec.Struct):
    data: Any

In [5]:
index = BPTree[MyKey, MyValue]()

In [6]:
faker.Faker.seed(1)
fake = faker.Faker("ru_RU")

In [7]:
data = []
gc.collect()

340

In [8]:
%%time
for i in tqdm(range(10_000)):
    random_int = fake.random_int(0, 99_999_999)
    val = f"{random_int:010} " + fake.name()
    result = index.insert(MyKey(val), fake.address())
    if result[0] is None:
        print(f"Duplicate key: {val}: {result[1]}")
    data.append(val)

  0%|          | 0/10000 [00:00<?, ?it/s]

CPU times: user 909 ms, sys: 23.4 ms, total: 932 ms
Wall time: 926 ms


In [9]:
index.validate(), gc.collect()

({}, 18)

In [10]:
index.find(MyKey('xxx'))

(None, None, None)

In [11]:
%%timeit
index.find(MyKey('0001677529 Шаров Леонтий Филатович'))

2.94 µs ± 65 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [12]:
index.values

['ст. Североморск, бул. Воровского, д. 81 к. 166, 439150',
 'клх Арсеньев, алл. Бабушкина, д. 9/4 стр. 4, 740681',
 'клх Арсеньев, алл. Бабушкина, д. 9/4 стр. 4, 740681',
 'д. Калачинск, наб. Тюленина, д. 1/8 к. 76, 585178',
 'с. Балашиха, бул. Льва Толстого, д. 92 к. 40, 685957',
 'с. Белый Яр (Томск.), ш. Радищева, д. 78 к. 6/7, 088995',
 'ст. Усть-Илимск, ул. Кирпичная, д. 107 к. 4, 192541',
 'клх Москва, МГУ, пр. Торговый, д. 646 к. 4/5, 483960',
 'д. Кетченеры, ш. Курчатова, д. 18 стр. 6/7, 423041',
 'г. Ухта, пер. Тамбовский, д. 3/9, 635139',
 'к. Темрюк, ул. 40 лет Победы, д. 5/1 к. 63, 563416',
 'клх Бавлы, алл. Краснодарская, д. 48, 598455',
 'к. Аша, ул. Максима Горького, д. 62 к. 799, 983914',
 'к. Всеволожск, ш. Калинина, д. 290, 103396',
 'с. Уренгой, пер. Прудный, д. 6/2 к. 15, 049576',
 'к. Кашира, ш. Шаумяна, д. 5 стр. 48, 335141',
 'г. Мценск, наб. Лермонтова, д. 25, 330361',
 'ст. Новороссийск, пр. Сенной, д. 285 к. 32, 541894',
 'ст. Юрюзань, пер. Высокий, д. 74 стр.

In [47]:
index.print_node(index.tree[index.root_node])


node-idx:1, prev:None, next:None, level: 1  <============ *ROOT*
   |-> MyKey(full_name='0001778656 Максимов Гаврила Измаилович'): 0, 48
   |-> MyKey(full_name='0003358040 Пахомов Эрнест Давидович'): 48, 25
   |-> MyKey(full_name='0004631176 Марина Андреевна Шарапова'): 25, 50
   |-> MyKey(full_name='0006047413 Лыткина Милица Оскаровна'): 50, 13
   |-> MyKey(full_name='0007915242 Савва Архипович Панов'): 13, 37
   |-> MyKey(full_name='0009772586 Екатерина Вячеславовна Панфилова'): 37, 20
   |-> MyKey(full_name='0011622887 Варвара Валентиновна Зимина'): 20, 41
   |-> MyKey(full_name='0013332303 Гаврилов Прокл Феликсович'): 41, 8
   |-> MyKey(full_name='0014595629 Зинаида Алексеевна Крылова'): 8, 59
   |-> MyKey(full_name='0016137119 Кондратьев Кузьма Ильич'): 59, 27
   |-> MyKey(full_name='0017703271 Брагина Василиса Никифоровна'): 27, 49
   |-> MyKey(full_name='0019146274 Демид Ермилович Ефимов'): 49, 14
   |-> MyKey(full_name='0020487853 Горшкова Надежда Богдановна'): 14, 51
   |-> M

In [48]:
index.min()[1]

MyKey(full_name='0000017833 Зайцева Валерия Ефимовна')

In [49]:
index.max()[1]

MyKey(full_name='0099994480 Евфросиния Рудольфовна Кузнецова')

In [50]:
data[0:10]

['0018034063 Брагина Ангелина Ждановна',
 '0002996023 Потапов Авксентий Тарасович',
 '0024951916 Владимирова Кира Васильевна',
 '0014486288 Регина Никифоровна Фомина',
 '0036142079 Кошелева Агата Натановна',
 '0061491422 Брагина Елизавета Вячеславовна',
 '0022477484 Лобанов Давыд Егорович',
 '0030249188 Куликов Анатолий Владиславович',
 '0010260429 Лаврентий Владиленович Аксенов',
 '0090483285 Феврония Георгиевна Горшкова']

In [51]:
random.shuffle(data)

In [53]:
%%time
for i, k in enumerate(data):
    ptr = index.find(MyKey(k))[0]
    if ptr is not None:
        print(f"{i:09} | found: {ptr} -> {index.keys[ptr]}, {index.values[ptr]}")
    else:
        ptr = index.find(k)
        print(f"Key: {k} not found")

print("Done!")

000000000 | found: 6911 -> MyKey(full_name='0022721402 Онуфрий Феоктистович Анисимов'), д. Медногорск, бул. Советской Армии, д. 708 к. 5, 205623
000000001 | found: 709 -> MyKey(full_name='0038105801 Мартын Игоревич Осипов'), г. Сухиничи, пр. Красных Партизан, д. 9/8, 760061
000000002 | found: 5904 -> MyKey(full_name='0008479539 Лапина Василиса Харитоновна'), клх Улан-Удэ, бул. Труда, д. 5 стр. 7/7, 859902
000000003 | found: 828 -> MyKey(full_name='0084464972 Акулина Александровна Крылова'), д. Тюмень, ул. Дружбы, д. 87 стр. 1, 415107
000000004 | found: 2094 -> MyKey(full_name='0093971436 Морозов Радован Эдуардович'), с. Североморск, ш. Деповское, д. 8, 907396
000000005 | found: 3808 -> MyKey(full_name='0075547999 Маслова Зинаида Юрьевна'), г. Томпа, ул. Волкова, д. 7, 025419
000000006 | found: 3926 -> MyKey(full_name='0099529024 Вероника Робертовна Селиверстова'), г. Карабулак, пр. Ягодный, д. 13, 297087
000000007 | found: 8693 -> MyKey(full_name='0016604825 Полякова Анжелика Дмитриевн